In [7]:
%run Requirements.ipynb

In [8]:
%run Configuration.ipynb

Extracting Data directly from Kaggle and Storing it (Dynamic)

In [9]:
file_path = os.path.join(target_folder, file_name)
if os.path.exists(file_path):
    os.remove(file_path)

kaggle.api.dataset_download_files(dataset_name, path=target_folder, unzip=True)

df = pd.read_csv(file_path)

Dataset URL: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020


Setting-Up MySQL Connector

In [10]:
conn = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    password='ronitgupta28',
    database='formulaOne'
)
cursor = conn.cursor()

Staging the Raw data directly from CSV to DB

In [11]:
file_paths = File_path

for file_name, file_path in file_paths.items():
    df = pd.read_csv(file_path)

    columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])

    drop_table_query = f"DROP TABLE IF EXISTS {file_name};"
    cursor.execute(drop_table_query)

    create_table_query = f"CREATE TABLE IF NOT EXISTS {file_name} ({columns});"
    cursor.execute(create_table_query)

    for index, row in df.iterrows():
        values = tuple(row.values)
        insert_query = f"REPLACE INTO {file_name} VALUES {values};"
        insert_query = insert_query.replace(", nan", ", NULL")
        cursor.execute(insert_query)

conn.commit()

In [12]:
file_paths = File_path

for file_name, file_path in file_paths.items():
    df = spark.read.csv(file_path, header=True)
    df.write.mode('overwrite').format('parquet').options(header=True).save("StagingData/" + file_name)